# Retrieval

Retrieval is a common technique chatbots use to augment their responses with data outside a chat model's training data. This section will cover how to implement retrieval in the context of chatbots, but it's worth noting that retrieval is a very subtle and deep topic - we encourage you to explore [other parts of the documentation](/docs/use_cases/question_answering/) that go into greater depth!

## Setup

You'll need to install a few packages, and have your OpenAI API key set as an environment variable named `OPENAI_API_KEY`:

In [1]:
pip install sentence_transformers langchain langchain-openai chromadb beautifulsoup4 unstructured

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

Let's also set up a chat model that we'll use for the below examples.

In [3]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
chat = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0.2)

## Creating a retriever

We'll use [the LangSmith documentation](https://docs.smith.langchain.com/overview) as source material and store the content in a vectorstore for later retrieval. Note that this example will gloss over some of the specifics around parsing and storing a data source - you can see more [in-depth documentation on creating retrieval systems here](/docs/use_cases/question_answering/).

Let's use a document loader to pull text from the docs:

In [5]:
#from langchain_community.document_loaders import WebBaseLoader
#
#loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
#data = loader.load()

In [5]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('../docs', glob="**/*.md")
data = loader.load()
len(data)

2

Next, we split it into smaller chunks that the LLM's context window can handle and store it in a vector database:

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter,MarkdownHeaderTextSplitter

#model = "paraphrase-multilingual-MiniLM-L12-v2"
#text_splitter = SentenceTransformersTokenTextSplitter(model_name=model,tokens_per_chunk=50,chunk_overlap=10)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

39


In [12]:
all_splits

[Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': '../docs/index.md'}),
 Document(page_content='Telekom MMS', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Über 2.200 Mitarbeitende an neun Standorten in Deutschland entwickeln zukunftsfähige Geschäftsmodelle für digitale Erlebnisse. Telekom MMS bietet ein dynamisches Web- und Application-Management und', metadata={'source': '../docs/mms.md'}),
 Document(page_content='und sorgt mit dem hauseigenen akkreditierten Test-Center für höchste Softwarequalität, Barrierefreiheit und IT-Sicherheit.', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Das Unternehmen zählt zum Konzern Deutsche Telekom AG und tritt ebenfalls mit 

Then we embed and store those chunks in a vector database:

In [13]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

#default_ef = embedding_functions.DefaultEmbeddingFunction()

#modell = "all-MiniLM-L6-v2"
model = "paraphrase-multilingual-MiniLM-L12-v2"
#modell = "sentence-transformers/LaBSE"
#modell = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

embedding_function = SentenceTransformerEmbeddings(model_name=model)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding_function)#, embedding=OpenAIEmbeddings())


In [14]:
vectorstore.similarity_search("Geschäftsleitung",k=5)

[Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens.', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': '../docs/index.md'}),
 Document(page_content='Geschichte', metadata={'source': '../docs/mms.md'}),
 Document(page_content='und IT-Sicherheit.', metadata={'source': '../docs/mms.md'})]

And finally, let's create a retriever from our initialized vectorstore:

In [15]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

docs = retriever.invoke("Telekom MMS")

docs

[Document(page_content='Telekom MMS', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': '../docs/mms.md'}),
 Document(page_content='die bisherigen Tochtergesellschaften der Telekom DeTeSystems, DeTeCSM, Detecon, T-Data, T-Nova, die Konzernbereiche Datenkommunikation und Media Broadcast sowie das von Daimler erworbene debis', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Im März 2023 ging das Unternehmen zurück an die Deutsche Telekom AG und firmiert nun unter Deutsche Telekom MMS GmbH. Dadurch rückt das Unternehmen näher an die mittelständischen Geschäftskunden der', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Geschäftskunden der Telekom Deutschland heran und kann den Konzern auf dem Weg zur „Leading Digital Telko“ unterstü

We can see that invoking the retriever above results in some parts of the LangSmith docs that contain information about testing that our chatbot can use as context when answering questions. And now we've got a retriever that can return related data from the LangSmith docs!

## Document chains

Now that we have a retriever that can return LangChain docs, let's create a chain that can use them as context to answer questions. We'll use a `create_stuff_documents_chain` helper function to "stuff" all of the input documents into the prompt. It will also handle formatting the docs as strings.

In addition to a chat model, the function also expects a prompt that has a `context` variables, as well as a placeholder for chat history messages named `messages`. We'll create an appropriate prompt and pass it as shown below:

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

We can invoke this `document_chain` by itself to answer questions. Let's use the docs we retrieved above and the same question, `how can langsmith help with testing?`:

In [17]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Was macht die Telekom MMS?")
        ],
    }
)

'Die Telekom MMS ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Nach der Rückkehr an die Deutsche Telekom AG firmiert das Unternehmen nun unter Deutsche Telekom MMS GmbH und kann den Konzern auf dem Weg zur „Leading Digital Telko“ unterstützen.'

Looks good! For comparison, we can try it with no context docs and compare the result:

In [25]:
document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content="Was ist die Telekom MMS?")
        ],
    }
)

'Die Telekom MMS steht für "Multimedia Messaging Service" und ist ein Dienst, der es ermöglicht, Multimedia-Inhalte wie Fotos, Videos und Audio-Dateien über das Mobilfunknetz zu versenden. MMS-Nachrichten können von einem Mobiltelefon an ein anderes gesendet werden und bieten eine Möglichkeit, multimediale Inhalte mit anderen zu teilen.'

We can see that the LLM does not return any results.

## Retrieval chains

Let's combine this document chain with the retriever. Here's one way this can look:

In [19]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

Given a list of input messages, we extract the content of the last message in the list and pass that to the retriever to fetch some documents. Then, we pass those documents as context to our document chain to generate a final response.

Invoking this chain combines both steps outlined above:

In [20]:
retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Wer ist die Gechäftsführerin der MMS?")
        ],
    }
)

{'messages': [HumanMessage(content='Wer ist die Gechäftsführerin der MMS?')],
 'context': [Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': '../docs/mms.md'}),
  Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': '../docs/index.md'}),
  Document(page_content='und IT-Sicherheit.', metadata={'source': '../docs/mms.md'}),
  Document(page_content='Indem T-Systems MMS eine künstliche Intelligenz mit einer weiblichen Gesangsstimme trainierte, war das Unternehmen 2022 maßgeblich an der weltweit ersten KI-Oper „chasing waterfalls“ beteiligt, die', metadata={'source': '../docs/mms.md'}),
  Document(page_content='2018 schloss T-Systems MMS einen Kooperationsvertrag mit dem Smart Systems Hub und ist seitdem einer der strategischen Partner. Mit dem Ziel, die reale und die digitale Welt zu vernetzen, ermöglicht', metadata={'source': '../docs/mms.md'})],
 'answer': 'Die Information über die Geschäftsführerin der MMS ist 

Looks good!

## Query transformation

Our retrieval chain is capable of answering questions about LangSmith, but there's a problem - chatbots interact with users conversationally, and therefore have to deal with followup questions.

The chain in its current form will struggle with this. Consider a followup question to our original question like `Tell me more!`. If we invoke our retriever with that query directly, we get documents irrelevant to LLM application testing:

In [21]:
retriever.invoke("Tell me more!")

[Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': '../docs/mms.md'}),
 Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': '../docs/index.md'}),
 Document(page_content='Geschichte', metadata={'source': '../docs/mms.md'}),
 Document(page_content='und IT-Sicherheit.', metadata={'source': '../docs/mms.md'}),
 Document(page_content='über Content-, Test- und Support-Services bis zur Entwicklung von Software. Dazu zählen Web- und Applikations-Management und das Test and Integration Center für Softwarequalität, Barrierefreiheit und', metadata={'source': '../docs/mms.md'})]

This is because the retriever has no innate concept of state, and will only pull documents most similar to the query given. To solve this, we can transform the query into a standalone query without any external references an LLM.

Here's an example:

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transformation_chain = query_transform_prompt | chat

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Wer ist der Gechäftsführer der MMS?"),
            #AIMessage(
            #    content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            #),
            #HumanMessage(content="Tell me more!"),
        ],
    }
)

AIMessage(content='"MMS Geschäftsführer"', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 55, 'total_tokens': 63}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_68a7d165bf', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}})

Awesome! That transformed query would pull up context documents related to LLM application testing.

Let's add this to our retrieval chain. We can wrap our retriever as follows:

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 0,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

Then, we can use this query transformation chain to make our retrieval chain better able to handle such followup questions:

In [24]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

Awesome! Let's invoke this new chain with the same inputs as earlier:

In [65]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Wer ist der Geschäftsführer der MMS?"),
        ]
    }
)

{'messages': [HumanMessage(content='Wer ist der Geschäftsführer der MMS?')],
 'context': [Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='spitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens. Ehemalige Geschäftsführer sind: Im Jahr 2005 initiierte das Unternehmen das Dresdner Zukunftsforum zum Thema „Arbeiten und Leben in der digitalen Welt“, das seither fünfmal stattgefunden hat. Im Jahr 2006 war T-Systems Multimedia Solutions sowohl Preisträger des Ludwig-Erhard-Pre', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Gewinnern des Outstanding Security Performance Awards.

In [27]:
import langchain
langchain.debug = True

conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Was ist die Telekom MMS?"),
            AIMessage(
                content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
            ),
            HumanMessage(content="Erzähl mir mehr!"),
        ],
    }
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<context>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:chat_retriever_chain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:chat_retriever_chain > 5:chain:RunnableLambda] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:chat_retriever_chain > 5:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": false
}
[chain/start] [1:chain:RunnableSequence > 2

{'messages': [HumanMessage(content='Was ist die Telekom MMS?'),
  AIMessage(content='Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management.'),
  HumanMessage(content='Erzähl mir mehr!')],
 'context': [Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': '../docs/mms.md'}),
  Document(page_content='Telekom MMS', metadata={'source': '../docs/mms.md'}),
  Document(page_content='Geschäftskunden der Telekom Deutschland heran und kann den Konzern auf dem Weg zur „Leading Digital Telko“ unterstützen.', metadata={'source': '../docs/mms.md'}),
  Document(page_co

In [23]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    data = conversational_retrieval_chain.invoke(
        {
            "messages": [
                HumanMessage(content="Was ist die Telekom MMS?"),
                AIMessage(
                    content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
                ),
                HumanMessage(content="Erzähl mir mehr!"),
            ],
        }
    )
    print(data)
    print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")

{'messages': [HumanMessage(content='Was ist die Telekom MMS?'), AIMessage(content='Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management.'), HumanMessage(content='Erzähl mir mehr!')], 'context': [Document(page_content='Telekom MMS', metadata={'source': 'docs/mms.md'}), Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': 'docs/mms.md'}), Document(page_content='Im März 2023 ging das Unternehmen zurück an die Deutsche Telekom AG und firmiert nun unter Deutsche Telekom MMS GmbH. Dadurch rückt das Unternehmen näher an die mittelständischen Geschäftskunden d

You can check out [this LangSmith trace](https://smith.langchain.com/public/bb329a3b-e92a-4063-ad78-43f720fbb5a2/r) to see the internal query transformation step for yourself.

## Streaming

Because this chain is constructed with LCEL, you can use familiar methods like `.stream()` with it:

In [25]:
stream = conversational_retrieval_chain.stream(
    {
            "messages": [
                HumanMessage(content="Was ist die Telekom MMS?"),
                AIMessage(
                    content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
                ),
                HumanMessage(content="Erzähl mir mehr!"),
            ],
    }
)

for chunk in stream:
    print(chunk)

{'messages': [HumanMessage(content='Was ist die Telekom MMS?'), AIMessage(content='Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management.'), HumanMessage(content='Erzähl mir mehr!')]}
{'context': [Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': 'docs/mms.md'}), Document(page_content='Telekom MMS', metadata={'source': 'docs/mms.md'}), Document(page_content='Geschäftskunden der Telekom Deutschland heran und kann den Konzern auf dem Weg zur „Leading Digital Telko“ unterstützen.', metadata={'source': 'docs/mms.md'}), Document(page_content='Über 2.200 M

## Further reading

This guide only scratches the surface of retrieval techniques. For more on different ways of ingesting, preparing, and retrieving the most relevant data, check out [this section](/docs/modules/data_connection/) of the docs.